In [27]:
import numpy as np
import pandas as pd

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import stanza

## Load Data and Models

### Tweet Data

In [39]:
### add code to load our dataset
file_name = "exp1.csv"
df = pd.read_csv(file_name)

In [40]:
df

,ID,original_ID,tweet,sarcastic,rephrase
0,17618,test_a_727,I love my friends,0,NaN
1,13604,train_921,i love shoegaze sm,0,NaN
2,19730,train_447,I hate you so much @Eagles,1,Being an Eagles fan is disappointing
3,14998,train_3171,i hate being one of the boys bc one of them ju...,0,NaN
4,719,train_2204,I hate this guy so much https://t.co/lHPKXjNLL4,0,NaN
5,10166,train_3320,i love the word junction,0,NaN
6,9087,test_a_757,I love a Monday morning so glad the weekends o...,1,NaN
7,19559,train_2771,i love my life,0,NaN
8,6869,train_2810,i love my friends,0,NaN
9,5755,train_2464,I love me some cupcakes! Especially Sprinkles....,0,NaN


### Sentiment Classifier

In [6]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def classify_sentiment(text) :
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    # [ negative, neutral, positive ]
    return scores

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Constituency Parser

In [7]:
parser = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via th

## Sarcasm Classifier

In [35]:
def sentiment_distance(tweet, verb, noun_phrase):
  """Calculates the sentiment distance (euclidic distance) between the sentiment scores of the V and NP of a given tweet."""

  tweet_sentiment = classify_sentiment(tweet)
  tweet_no_v_sentiment = classify_sentiment(tweet.replace(verb, "")) ##TO DO: change (ask Samba)
  tweet_no_np_sentiment = classify_sentiment(tweet.replace(noun_phrase, "")) ##TO DO: change (ask Samba)
  v_sentiment = tweet_sentiment - tweet_no_v_sentiment
  np_sentiment = tweet_sentiment - tweet_no_np_sentiment

  return np.linalg.norm(v_sentiment - np_sentiment)

def is_sarc(sentiment_dist, threshold):
  """Returns True if the sentiment distance is greater than a given threshold and False otherwise."""

  return sentiment_dist > threshold

## Define Threshold

In [42]:
def define_threshold(tweets, gold_annotations, threshold_list):
  """
  Creates a list of tuples containing 1) the tweet body, 2) the sarc/non-sarc gold annotation, 3) the sentiment distance between V and NP.
  Then, iterates over a list of thresholds and for each threshold calculates the accuracy between the gold annotations and the predicted values.
  Returns a dictionary of thresholds and accuracies for those thresholds.
  """

  tweet_annotation_sentiment_distance = []
  for tweet, annotation in zip(tweets, gold_annotations):
    tweet_split = tweet.split()
    verb = tweet_split[1]
    noun_phrase = " ".join(tweet_split[2:])
    tweet_annotation_sentiment_distance.append((tweet, annotation, sentiment_distance(tweet, verb, noun_phrase)))

  # scaling of the data
  sentiment_max = max([x[2] for x in tweet_annotation_sentiment_distance])
  tweet_annotation_sentiment_distance = [(t,a, sentiment_dist/sentiment_max) for t, a, sentiment_dist in tweet_annotation_sentiment_distance]

  threshold_accuracy_dict = {}
  for threshold in threshold_list:
    correct = 0
    for _, annotation, sentiment_dist in tweet_annotation_sentiment_distance:
      if is_sarc(sentiment_dist, threshold):
        correct += 1

    threshold_accuracy_dict[threshold] = correct / len(tweet_annotation_sentiment_distance)

  return threshold_accuracy_dict

In [43]:
gap = 0.001
threshold_list = list(np.arange(0, 1 + gap, gap))

threshold_accuracy_dict = define_threshold(df["tweet"], df["sarcastic"], threshold_list)
print(max(threshold_accuracy_dict, key=threshold_accuracy_dict.get))

0.0


In [45]:
threshold_accuracy_dict

{0.0: 1.0,
 0.001: 1.0,
 0.002: 1.0,
 0.003: 1.0,
 0.004: 1.0,
 0.005: 1.0,
 0.006: 1.0,
 0.007: 1.0,
 0.008: 1.0,
 0.009000000000000001: 1.0,
 0.01: 1.0,
 0.011: 1.0,
 0.012: 1.0,
 0.013000000000000001: 1.0,
 0.014: 1.0,
 0.015: 1.0,
 0.016: 1.0,
 0.017: 1.0,
 0.018000000000000002: 1.0,
 0.019: 1.0,
 0.02: 1.0,
 0.021: 1.0,
 0.022: 1.0,
 0.023: 1.0,
 0.024: 1.0,
 0.025: 1.0,
 0.026000000000000002: 1.0,
 0.027: 1.0,
 0.028: 1.0,
 0.029: 1.0,
 0.03: 1.0,
 0.031: 1.0,
 0.032: 1.0,
 0.033: 1.0,
 0.034: 1.0,
 0.035: 1.0,
 0.036000000000000004: 1.0,
 0.037: 1.0,
 0.038: 1.0,
 0.039: 1.0,
 0.04: 1.0,
 0.041: 1.0,
 0.042: 1.0,
 0.043000000000000003: 1.0,
 0.044: 1.0,
 0.045: 1.0,
 0.046: 1.0,
 0.047: 1.0,
 0.048: 1.0,
 0.049: 1.0,
 0.05: 1.0,
 0.051000000000000004: 1.0,
 0.052000000000000005: 1.0,
 0.053: 1.0,
 0.054: 1.0,
 0.055: 1.0,
 0.056: 1.0,
 0.057: 1.0,
 0.058: 1.0,
 0.059000000000000004: 1.0,
 0.06: 1.0,
 0.061: 1.0,
 0.062: 1.0,
 0.063: 1.0,
 0.064: 1.0,
 0.065: 1.0,
 0.066: 1.0,
 0

## Other Tests

In [ ]:
doc = parser('I just love crowded buses.\n\She gave me a gift. Yay I love crowded buses.')
for i, sentence in enumerate(doc.sentences):
    print(sentence.constituency)

(ROOT (S (NP (PRP I)) (ADVP (RB just)) (VP (VBP love) (NP (JJ crowded) (NNS buses))) (. .)))
(ROOT (S (NFP \) (NP (PRP She)) (VP (VBD gave) (NP (PRP me)) (NP (DT a) (NN gift))) (. .)))
(ROOT (S (NP (UH Yay)) (NP (PRP I)) (VP (VBP love) (NP (JJ crowded) (NNS buses))) (. .)))


In [ ]:
classify_sentiment("I love when people are loud.")

[-1.6430142  -0.38684654  2.1511881 ]


In [ ]:
classify_sentiment("I when people are loud.")

In [ ]:
classify_sentiment("I love.")

array([-2.0753474 , -0.19896051,  1.9287738 ], dtype=float32)